In [50]:
%tensorflow_version 2.x

In [51]:
import numpy as np
import keras.backend as K
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
import pandas as pd

### Import file

In [52]:
file_name = 'text.txt'
corpus = open(file_name).readlines()
#print(corpus)

### Data preprocessing


In [53]:
corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

# Remove punctuation in text and fit tokenizer on entire corpus

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
#print(corpus)

corpus = tokenizer.texts_to_sequences(corpus) # Convert text to sequence of integer values
n_samples = sum(len(s) for s in corpus)       # Total number of words in the corpus
V = len(tokenizer.word_index) + 1             # Total number of unique words in the corpus

In [54]:
#print(n_samples,V)

In [55]:
# Example of how word to integer mapping looks like in the tokenizer
#print(list((tokenizer.word_index.items())))

In [56]:
# Parameters
window_size = 2 
window_size_corpus = 4

# Set numpy seed for reproducible results
np.random.seed(42)

In [57]:
from keras.preprocessing import sequence

# Prepare the data for the CBOW model
def generate_data_cbow(corpus, window_size, V):
    all_in = []
    all_out = []

    # Iterate over all sentences
    for sentence in corpus:
        L = len(sentence)
        for index, word in enumerate(sentence):
            start = index - window_size
            end = index + window_size + 1

            # Empty list which will store the context words
            context_words = []
            for i in range(start, end):
                # Skip the 'same' word
                if i != index:
                    # Add a word as a context word if it is within the window size
                    if 0 <= i < L:
                        context_words.append(sentence[i])
                    else:
                        # Pad with zero if there are no words 
                        context_words.append(0)
            # Append the list with context words
            all_in.append(context_words)

            # Add one-hot encoding of the target word
            all_out.append(to_categorical(word, V))
                 
    return (np.array(all_in), np.array(all_out))

In [58]:
# Create the training data

X_cbow, y_cbow = generate_data_cbow(corpus, window_size, V)
#print('X_cbow = ',X_cbow,'\ny_cbow = ',y_cbow)


In [59]:
# Create the CBOW architecture
dim = 10

#for dim in dims:
cbow = Sequential()

    # Add an Embedding layer
cbow.add(Embedding(input_dim=V, 
                   output_dim=dim,
                   input_length = window_size*2, # Note that we now have 2L words for each input entry
                   embeddings_initializer='glorot_uniform'))

cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim, )))

cbow.add(Dense(V, activation='softmax', kernel_initializer='glorot_uniform'))

cbow.compile(optimizer=keras.optimizers.Adam(),
             loss='categorical_crossentropy',
             metrics=['accuracy'])
    
cbow.summary()
print("")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 10)             1700      
_________________________________________________________________
lambda_2 (Lambda)            (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 170)               1870      
Total params: 3,570
Trainable params: 3,570
Non-trainable params: 0
_________________________________________________________________



In [60]:
# Train CBOW model
cbow.fit(X_cbow, y_cbow, batch_size=64, epochs=500, verbose=1)
print("")

Epoch 1/500
5/5 [==============================] - 1s 4ms/step - loss: 5.1365 - accuracy: 0.0157
Epoch 2/500
5/5 [==============================] - 0s 4ms/step - loss: 5.1324 - accuracy: 0.0189
Epoch 3/500
5/5 [==============================] - 0s 3ms/step - loss: 5.1290 - accuracy: 0.0220
Epoch 4/500
5/5 [==============================] - 0s 4ms/step - loss: 5.1257 - accuracy: 0.0220
Epoch 5/500
5/5 [==============================] - 0s 4ms/step - loss: 5.1223 - accuracy: 0.0252
Epoch 6/500
5/5 [==============================] - 0s 4ms/step - loss: 5.1190 - accuracy: 0.0346
Epoch 7/500
5/5 [==============================] - 0s 4ms/step - loss: 5.1156 - accuracy: 0.0377
Epoch 8/500
5/5 [==============================] - 0s 4ms/step - loss: 5.1123 - accuracy: 0.0377
Epoch 9/500
5/5 [==============================] - 0s 4ms/step - loss: 5.1090 - accuracy: 0.0346
Epoch 10/500
5/5 [==============================] - 0s 4ms/step - loss: 5.1056 - accuracy: 0.0440
Epoch 11/500
5/5 [===========

In [61]:
# Save embeddings for vectors of length 50, 150 and 300 using cbow model
weights = cbow.get_weights()

# Get the embedding matrix
embedding = weights[0]
#print(embedding)

In [62]:
def add_0():
  words = list((tokenizer.word_index.items()))
  words.insert(0,('unkown',0))
  return words

In [63]:
#print(add_0())

In [64]:
def embed(word, embedding, vocab_size=V, tokenizer=tokenizer):
    int_word = tokenizer.texts_to_sequences([word])[0]
    bin_word = to_categorical(int_word, V)
    return np.dot(bin_word, embedding)

In [65]:
#print(embed('amnesia',embedding))

In [66]:
def compute_distance(word_a, word_b):
    
    point1 = embed(word_a,embedding)
    point2 = embed(word_b,embedding)
    
    return np.linalg.norm(point1 - point2)

In [67]:
def know_word(y_cbow):
  for i in range(len(y_cbow)):
    if y_cbow[i] == 1 :
      break
  return words[i][0]  

In [68]:
#print(know_word(y_cbow[0]))

In [69]:
def output(input):
  input_tokens = tokenizer.texts_to_sequences(input)
  #print(input_tokens)
  for i in range(len(cbow.predict(input_tokens)[0])):
    if cbow.predict(input_tokens)[0][i] == cbow.predict(input_tokens)[0].max() :
      break
  return words[i][0]

In [70]:
input_words = [input("Enter the neighbouring words (separated by spaces) :")]

Enter the neighbouring words (separated by spaces) :inability to new information


In [71]:
input_words

['inability to new information']

In [72]:
output(input_words)

'retrieve'